#### Query the parquet file on s3 
- It uses s3 select with specific SQL query
- Doc: `https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/s3/client/select_object_content.html#`
- Tutorial: `https://aws.amazon.com/blogs/storage/querying-data-without-servers-or-databases-using-amazon-s3-select/`

In [3]:
## libraries

import os
# import boto3
import duckdb

##### duckdb example

In [11]:
## file
input_file = '/home/ec2-user/projects/omics_analysis/scripts/file_tpm_pheno_gene_pqt.parquet'

## query
query = """
        SELECT * 
        FROM parquet_scan('{}')
        LIMIT 10
        """.format(input_file)


query2 = """
        SELECT tpm, gene, _study, _primary_site
        FROM parquet_scan('{}')
        WHERE gene = 'STT3A' AND _study = 'GTEX'
        """.format(input_file)
        
        


In [8]:
# connect to duckdb
con = duckdb.connect(database=':memory:', read_only=False)

# execute query
df = con.execute(query).fetchdf()

# print results
print(df)

               sample                 variable     tpm  \
0   ENSG00000242268.2  GTEX-S4Q7-0003-SM-3NM8M -3.4580   
1   ENSG00000259041.1  GTEX-S4Q7-0003-SM-3NM8M -9.9658   
2   ENSG00000270112.3  GTEX-S4Q7-0003-SM-3NM8M -3.6259   
3  ENSG00000167578.16  GTEX-S4Q7-0003-SM-3NM8M  4.5988   
4   ENSG00000278814.1  GTEX-S4Q7-0003-SM-3NM8M -9.9658   
5   ENSG00000078237.5  GTEX-S4Q7-0003-SM-3NM8M  2.2633   
6   ENSG00000269416.5  GTEX-S4Q7-0003-SM-3NM8M -6.5064   
7   ENSG00000263642.1  GTEX-S4Q7-0003-SM-3NM8M -9.9658   
8  ENSG00000146083.11  GTEX-S4Q7-0003-SM-3NM8M  5.7811   
9  ENSG00000158486.13  GTEX-S4Q7-0003-SM-3NM8M -4.2934   

                     detailed_category            primary disease or tissue  \
0  Cells - Ebv-Transformed Lymphocytes  Cells - Ebv-Transformed Lymphocytes   
1  Cells - Ebv-Transformed Lymphocytes  Cells - Ebv-Transformed Lymphocytes   
2  Cells - Ebv-Transformed Lymphocytes  Cells - Ebv-Transformed Lymphocytes   
3  Cells - Ebv-Transformed Lymphocytes  Cells

In [12]:
## query2
df2 = con.execute(query2).fetchdf()

# print results
print(df2)

         tpm   gene _study _primary_site
0     6.4975  STT3A   GTEX         Blood
1     6.2952  STT3A   GTEX  Blood Vessel
2     4.4263  STT3A   GTEX         Brain
3     6.3733  STT3A   GTEX       Thyroid
4     5.3664  STT3A   GTEX      Pancreas
...      ...    ...    ...           ...
7857  5.7632  STT3A   GTEX     Pituitary
7858  5.9855  STT3A   GTEX     Esophagus
7859  5.8278  STT3A   GTEX         Nerve
7860  4.9942  STT3A   GTEX         Brain
7861  6.8567  STT3A   GTEX          Lung

[7862 rows x 4 columns]


##### s3 select example

In [6]:
s3 = boto3.client('s3')

resp = s3.select_object_content(
    Bucket='biodata-omics-data',
    Key='xena/processed/file_tpm_pheno_gene_pqt.parquet',
    ExpressionType='SQL',
    Expression="SELECT * FROM s3object s where s.\"Name\" = 'Jane'",
    InputSerialization = {'Parquet': {}},
    OutputSerialization = {'CSV': {}},
)

In [7]:
for event in resp['Payload']:
    if 'Records' in event:
        records = event['Records']['Payload'].decode('utf-8')
        print(records)
    elif 'Stats' in event:
        statsDetails = event['Stats']['Details']
        print("Stats details bytesScanned: ")
        print(statsDetails['BytesScanned'])
        print("Stats details bytesProcessed: ")
        print(statsDetails['BytesProcessed'])
        print("Stats details bytesReturned: ")
        print(statsDetails['BytesReturned'])

EventStreamError: An error occurred (ParquetUnsupportedCompressionCodec) when calling the SelectObjectContent operation: Currently S3 Select does not support Parquet objects with ZSTD compression codec. Please see the API documentation for more details.